<a href="https://colab.research.google.com/github/HirunaD/LangChain/blob/main/11_SQL_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Required Packages**

In [1]:
!pip install langchain_openai -q
!pip install langchain_community -q
!pip install langchain -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


**Create SQLite Database and Tables**

In [2]:
import sqlite3

In [3]:
connection = sqlite3.connect("school.db")

In [4]:
cursor = connection.cursor()

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS teachers (
    teacher_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    subject_id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS teaches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    teacher_id INTEGER,
    subject_id INTEGER,
    FOREIGN KEY (teacher_id) REFERENCES teachers(teacher_id),
    FOREIGN KEY (subject_id) REFERENCES subjects(subject_id)
)
""")

connection.commit()

**Insert Sample Data into Tables**

In [6]:
teachers = [
    ("Kamal", 42),
    ("Saman", 29),
    ("Pawan", 34)
]
cursor.executemany("INSERT INTO teachers (name, age) VALUES (?, ?)", teachers)

subjects = [
    ("Mathematics",),
    ("Science",),
    ("History",),
    ("English",)
]
cursor.executemany("INSERT INTO subjects (subject_name) VALUES (?)", subjects)

teaches = [
    (1, 1),  # Kamal teaches Mathematics
    (1, 2),  # Kamal teaches Science
    (2, 3),  # Saman teaches History
    (3, 4),  # Pawan teaches English
]
cursor.executemany("INSERT INTO teaches (teacher_id, subject_id) VALUES (?, ?)", teaches)

connection.commit()

In [7]:
cursor.execute("SELECT * FROM teachers")
teachers = cursor.fetchall()
print(teachers)

[(1, 'Kamal', 42), (2, 'Saman', 29), (3, 'Pawan', 34)]


In [8]:
cursor.execute("""SELECT t.name
FROM teachers t
JOIN teaches te ON t.teacher_id = te.teacher_id
JOIN subjects s ON te.subject_id = s.subject_id
WHERE s.subject_name = 'Mathematics';""")
teachers = cursor.fetchall()
print(teachers)

[('Kamal',)]


**Initialize LangChain SQL Database**

In [10]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [11]:
from langchain_community.utilities.sql_database import SQLDatabase

In [12]:
db = SQLDatabase.from_uri(f"sqlite:///school.db")

In [13]:
print(db.get_usable_table_names())

['subjects', 'teachers', 'teaches']


In [14]:
print(db.table_info)


CREATE TABLE subjects (
	subject_id INTEGER, 
	subject_name TEXT NOT NULL, 
	PRIMARY KEY (subject_id)
)

/*
3 rows from subjects table:
subject_id	subject_name
1	Mathematics
2	Science
3	History
*/


CREATE TABLE teachers (
	teacher_id INTEGER, 
	name TEXT NOT NULL, 
	age INTEGER NOT NULL, 
	PRIMARY KEY (teacher_id)
)

/*
3 rows from teachers table:
teacher_id	name	age
1	Kamal	42
2	Saman	29
3	Pawan	34
*/


CREATE TABLE teaches (
	id INTEGER, 
	teacher_id INTEGER, 
	subject_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(teacher_id) REFERENCES teachers (teacher_id), 
	FOREIGN KEY(subject_id) REFERENCES subjects (subject_id)
)

/*
3 rows from teaches table:
id	teacher_id	subject_id
1	1	1
2	1	2
3	2	3
*/


**Initialize LLM**

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # temperature of 0 means that the highest probability tokens are always selected

**Create SQL Generation Chain**

In [16]:
from langchain.chains import create_sql_query_chain

query_generate = create_sql_query_chain(llm, db)

**Setup SQL Execution Tool**

In [17]:
from langchain_community.tools import QuerySQLDatabaseTool

query_execute = QuerySQLDatabaseTool(db=db)

In [18]:
query = query_generate.invoke({"question": "Details of all teachers"})
print(query)

SELECT "teacher_id", "name", "age" FROM teachers


In [19]:
query_execute.invoke(query)

"[(1, 'Kamal', 42), (2, 'Saman', 29), (3, 'Pawan', 34)]"

In [20]:
query = query_generate.invoke({"question": "Which teachers are assigned to teach Mathematics?"})
print(query)

SELECT t.name
FROM teachers t
JOIN teaches te ON t.teacher_id = te.teacher_id
JOIN subjects s ON te.subject_id = s.subject_id
WHERE s.subject_name = "Mathematics"
LIMIT 5;


In [21]:
query_execute.invoke(query)

"[('Kamal',)]"

**Create Answer Generator Chain**

In [22]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given a user question, the generated SQL query, and its result, write a clear and natural answer to the question.

    User Question: {question}
    SQL Query: {query}
    SQL Result: {result}

    Final Answer:"""
)

final_answer = answer_prompt | llm | StrOutputParser()

In [23]:
chain = (
    RunnablePassthrough.assign(query=query_generate).assign(
        result=itemgetter("query") | query_execute
    )
    | final_answer
)

In [24]:
chain.invoke({"question": "Details of all teachers"})

'Here are the details of all teachers: \n1. Teacher ID: 1, Name: Kamal, Age: 42\n2. Teacher ID: 2, Name: Saman, Age: 29\n3. Teacher ID: 3, Name: Pawan, Age: 34'

In [25]:
chain.invoke({"question": "Which teachers are assigned to teach Mathematics?"})

'Kamal is the teacher assigned to teach Mathematics.'